In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os  # permet d'aller écrire dans les répertoires
from openpyxl import Workbook
from openpyxl.styles.fills import PatternFill, FILL_SOLID
import pandas as pd

In [42]:
option = webdriver.ChromeOptions()
option.add_argument("--incognito")  # Navigation privée
# option.add_argument("kiosk")  # grand ecran sur mac
#prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
#option.add_experimental_option('prefs', prefs)

In [43]:
# # Pour Mac
browser = webdriver.Chrome(
   executable_path="/Users/thomasbouiniere/Desktop/Scrapping/chromedriver", options=option)

# #Pour windows
# browser = webdriver.Chrome(
#     executable_path=r"C:/Users/HP/OneDrive/Bureau/scrap/chromedriver.exe", options=option)

<ipython-input-43-3bc954d34f69>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(


In [44]:
# ---------------------------------------------------------------
# Betclic
# ---------------------------------------------------------------
compets_betclic = {'ldc': 'https://www.betclic.fr/football-s1/ligue-des-champions-c8',
                   'pl': 'https://www.betclic.fr/football-s1/angl-premier-league-c3',
                   'l1': 'https://www.betclic.fr/football-s1/ligue-1-uber-eats-c4',
                   'liga': 'https://www.betclic.fr/football-s1/espagne-liga-primera-c7'}

res_betclic = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
               'pl': {'cotes': [], 'noms': [], 'dates': []},
               'l1': {'cotes': [], 'noms': [], 'dates': []},
               'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [45]:
for i, compet in enumerate(compets_betclic):
    browser.get(compets_betclic[compet])
    time.sleep(2)
    if i ==1:
        time.sleep(2)
        button = browser.find_element_by_xpath(
            '//*[@id="popin_tc_privacy_button_2"]')
        button.click()

    cotes_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/sports-events-event-markets/sports-markets-default[1]/div/sports-selections-selection/div[1]/span[2]')
    for x in cotes_temp:
        res_betclic[compet]['cotes'].append(x.text)


    nom_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/div/scoreboards-scoreboard/scoreboards-scoreboard-global/div/div/div')
    for x in nom_temp:
        res_betclic[compet]['noms'].append(x.text)

    date_temp = browser.find_elements_by_xpath(
        '/html/body/app-desktop/div[1]/div/bcdk-content-scroller/div/sports-competition/div/sports-events-list/bcdk-vertical-scroller/div/div[2]/div/div/div/div[2]/div/sports-events-event/a/div/sports-events-event-info/div')
    for x in date_temp:
        res_betclic[compet]['dates'].append(x.text)

<ipython-input-45-52f6c4760421>:10: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_temp = browser.find_elements_by_xpath(
<ipython-input-45-52f6c4760421>:16: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  nom_temp = browser.find_elements_by_xpath(
<ipython-input-45-52f6c4760421>:21: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  date_temp = browser.find_elements_by_xpath(
<ipython-input-45-52f6c4760421>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = browser.find_element_by_xpath(


In [46]:
for compet in res_betclic:
    for i,name in enumerate(res_betclic[compet]['noms']):
        if name =='Barcelone':
            res_betclic[compet]['noms'][i] = 'FC Barcelone'
        elif name == 'Séville':
            res_betclic[compet]['noms'][i] = 'FC Séville'
    res_betclic[compet]['cotes']= [[res_betclic[compet]['cotes'][i],res_betclic[compet]['cotes'][i+1],res_betclic[compet]['cotes'][i+2]] for i in range(len(res_betclic[compet]['cotes'])-2) if i%3==0]
    res_betclic[compet]['noms']=[[res_betclic[compet]['noms'][i],res_betclic[compet]['noms'][i+1]] for i in range(len(res_betclic[compet]['noms'])-1) if i%2 ==0]
print(res_betclic['ldc']['noms'])



[['Benfica', 'Liverpool'], ['Manchester City', 'Atletico Madrid'], ['Villarreal', 'Bayern Munich'], ['Chelsea', 'Real Madrid']]


In [91]:
for compet in res_betclic:
    for cote in res_betclic[compet]['cotes']:
        for i,x in enumerate(cote):
            if x =='':
                cote[i]='0'


1
1
1
1
1
1


In [92]:
print(res_betclic)


{'ldc': {'cotes': [['7,70', '4,30', '1,34'], ['1,34', '4,45', '7,25'], ['4,95', '4,00', '1,52'], ['0', '0', '0']], 'noms': [['Benfica', 'Liverpool'], ['Manchester City', 'Atletico Madrid'], ['Villarreal', 'Bayern Munich'], ['Chelsea', 'Real Madrid']], 'dates': ['05/04/2022 21:00', '05/04/2022 21:00', '06/04/2022 21:00', '06/04/2022 21:00']}, 'pl': {'cotes': [['1,11', '9,00', '18'], ['12', '6,25', '1,21'], ['1,33', '4,85', '8,90'], ['1,47', '4,15', '6,80'], ['2,88', '3,05', '2,52'], ['2,35', '3,60', '2,72'], ['1,49', '4,60', '5,55'], ['1,75', '3,70', '4,35'], ['1,47', '4,40', '6,20'], ['3,90', '3,40', '1,93'], ['2,60', '3,15', '2,72'], ['2,40', '3,00', '2,90'], ['4,25', '3,55', '1,75'], ['2,72', '3,15', '2,48'], ['4,75', '3,65', '1,66'], ['1,56', '3,75', '5,45'], ['3,10', '3,20', '2,20'], ['2,70', '3,05', '2,55'], ['2,15', '3,30', '3,10'], ['2,95', '3,15', '2,28']], 'noms': [['Liverpool', 'Watford'], ['Burnley', 'Manchester City'], ['Chelsea', 'Brentford'], ['Brighton', 'Norwich City'],

In [48]:
# ---------------------------------------------------------------
# Winamax
# ---------------------------------------------------------------

compets_winamax = {'ldc': 'https://www.winamax.fr/paris-sportifs/sports/1/800000542/23',
                   'pl': 'https://www.winamax.fr/paris-sportifs/sports/1/1/1',
                   'l1': 'https://www.winamax.fr/paris-sportifs/sports/1/7/4',
                   'liga': 'https://www.winamax.fr/paris-sportifs/sports/1/32/36'}

res_winamax = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
               'pl': {'cotes': [], 'noms': [], 'dates': []},
               'l1': {'cotes': [], 'noms': [], 'dates': []},
               'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [49]:
for i, compet in enumerate(compets_winamax):
    browser.get(compets_winamax[compet])
    time.sleep(1)

    '''On récupère les cotes'''

    cotes_liga_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div[1]/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a')
    cotes_liga = []
    try:
        for lien in cotes_liga_temp:
            for i in range(0, 3):
                x = lien.get_property('lastChild')
                x = x.get_property('lastChild')
                x = x.get_property('lastChild')
                x = x.get_property('children')[0]
                x = x.get_property('children')[i]
                x = x.get_property('children')[1]
                x = x.get_property('children')[0]
                x = x.get_property('children')[1]
                res_winamax[compet]['cotes'].append(x.text)
    except:
        None

    '''On récupère les noms d'équipes'''
    noms_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a/div[2]/div[1]/div[1]/div[1]/div/span')
    for x in noms_temp:
        res_winamax[compet]['noms'].append(x.text)

    '''On récupère les dates des matchs'''

    dates_temp = browser.find_elements_by_xpath(
        '//*[@id="app"]/div/div[1]/div/div[2]/div/section/div[3]/div[1]/div/div/div/div/div[1]/a/div[1]/div[2]/div')
    dates_temp = dates_temp[:-1]

    for x in dates_temp:
        res_winamax[compet]['dates'].append(x.text)

<ipython-input-49-64ccc6ec26ff>:7: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_liga_temp = browser.find_elements_by_xpath(
<ipython-input-49-64ccc6ec26ff>:26: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  noms_temp = browser.find_elements_by_xpath(
<ipython-input-49-64ccc6ec26ff>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dates_temp = browser.find_elements_by_xpath(


In [50]:
for compet in res_winamax: 
    while ' - ' in res_winamax[compet]['noms']:
        res_winamax[compet]['noms'].remove(' - ')
        
for compet in res_winamax:
    for i,name in enumerate(res_winamax[compet]['noms']):
        if name == 'CF Cadiz':
            res_winamax[compet]['noms'][i] = 'Cadix'

for compet in res_betclic:
    res_winamax[compet]['cotes']= [[res_winamax[compet]['cotes'][i],res_winamax[compet]['cotes'][i+1],res_winamax[compet]['cotes'][i+2]] for i in range(len(res_winamax[compet]['cotes'])-2) if i%3==0]
    res_winamax[compet]['noms']=[[res_winamax[compet]['noms'][i],res_winamax[compet]['noms'][i+1]] for i in range(len(res_winamax[compet]['noms'])-1) if i%2 ==0]

print(res_winamax)

{'ldc': {'cotes': [['1,40', '4,75', '8,50'], ['8,50', '4,80', '1,37'], ['2,05', '3,40', '3,75'], ['5,85', '4,30', '1,55']], 'noms': [['Manchester City', 'Atletico Madrid'], ['Benfica', 'Liverpool'], ['Chelsea', 'Real Madrid'], ['Villarreal', 'Bayern Munich']], 'dates': ['MARDI À 21:00', 'MARDI À 21:00', '6 AVR. 2022 À 21:00', '6 AVR. 2022 À 21:00']}, 'pl': {'cotes': [['1,11', '9,50', '21'], ['14,50', '7,00', '1,21'], ['1,34', '5,00', '10'], ['2,90', '3,15', '2,60'], ['1,50', '4,10', '6,75'], ['2,40', '3,65', '2,75'], ['1,51', '4,65', '6,25'], ['1,80', '3,80', '4,50'], ['1,52', '4,50', '6,25'], ['3,90', '3,40', '2,00'], ['2,60', '3,15', '2,70'], ['2,40', '3,05', '2,95'], ['4,30', '3,65', '1,76'], ['1,58', '3,80', '5,60'], ['4,80', '3,70', '1,68'], ['2,75', '3,20', '2,50'], ['3,15', '3,25', '2,20'], ['3,00', '3,20', '2,30'], ['2,70', '3,10', '2,55'], ['2,15', '3,30', '3,15'], ['2,05', '3,55', '3,25']], 'noms': [['Liverpool', 'Watford'], ['Burnley', 'Manchester City'], ['Chelsea', 'Brentf

In [51]:
# ---------------------------------------------------------------
# unibet
# ---------------------------------------------------------------

compets_unibet = {'ldc': 'https://www.unibet.fr/sport/football/ligue-des-champions/ligue-des-champions-matchs',
                  'pl': 'https://www.unibet.fr/sport/football/premier-league/matchs',
                  'l1': 'https://www.unibet.fr/sport/football/ligue-1-ubereats/ligue-1-matchs',
                  'liga': 'https://www.unibet.fr/sport/football/liga/matchs'}

res_unibet = {'ldc': {'cotes': [], 'noms': [], 'dates': []},
              'pl': {'cotes': [], 'noms': [], 'dates': []},
              'l1': {'cotes': [], 'noms': [], 'dates': []},
              'liga': {'cotes': [], 'noms': [], 'dates': []}}

In [52]:
for i, compet in enumerate(compets_unibet):
    browser.get(compets_unibet[compet])
    time.sleep(2)
    '''On cherche les côtes '''
    cotes_temp = browser.find_elements_by_xpath(
        '//*[@id="page__competitionview"]/div[1]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]')
    for match in cotes_temp:
        for i in range(0, 3):
            x = match.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[0]
            x = x.get_property('children')[i]
            x = x.get_property('children')[0]
            x = x.get_property('children')[2]
            res_unibet[compet]['cotes'].append(x.text)

    '''On cherche les noms d'équipes'''

    noms_temp = browser.find_elements_by_xpath(
        '//*[@id="page__competitionview"]/div[1]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div[2]/div[1]/div/span')
    for x in noms_temp:
        res_unibet[compet]['noms'].append(x.text.split('-'))

<ipython-input-52-e5da6e225fe2>:5: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cotes_temp = browser.find_elements_by_xpath(
<ipython-input-52-e5da6e225fe2>:20: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  noms_temp = browser.find_elements_by_xpath(


In [53]:
sites={'ldc': "https://www.livefootballtickets.com/champions-league-tickets.html" ,
        'lp': "https://www.livefootballtickets.com/english-championship-tickets.html",
        'liga':'https://www.livefootballtickets.com/la-liga-tickets.html',
        'l1':'https://www.livefootballtickets.com/ligue-1-tickets.html'}

k={'ldc':{},'lp':{},'liga':{},'l1':{}}
# option = webdriver.ChromeOptions()
# option.add_argument("--incognito") 
# browser = webdriver.Chrome(
#     executable_path=r"C:/Users/HP/OneDrive/Bureau/scrap/chromedriver.exe", options=option)
# for compet in sites:
#     browser.get(sites[compet])
#     liste = browser.find_elements_by_xpath(
#         '/html/body/div[4]/form/div[3]/div[1]/div[1]/div/div[3]/ol/li')

#     for i in range(len(liste)):
#         print(i)
#         x=liste[i]
#         x=x.get_property('children')
#         x=x[0].text.split('\n')
#         try:
#             k[compet][x[4]]={}
#             k[compet][x[4]]['date']= x[1]+' '+x[0]+' '+x[2]
#             k[compet][x[4]]['hour'],k[compet][x[4]]['place']= x[5].split(' - ')[0],x[5].split(' - ')[1]
#             k[compet][x[4]]['equip1'],k[compet][x[4]]['equip2']= x[4].split(' v ')[0],x[4].split(' v ')[1]
#         except:
#             pass
        

        
#         try:
#             tick = browser.find_element_by_xpath('/html/body/div[4]/form/div[3]/div[1]/div[1]/div/div[3]/ol/li['+str(i+1)+']/div/div[3]/a')
#             browser.get(tick.get_attribute('href'))
#             #available=browser.find_elements_by_xpath('/html/body/div[4]/form/div[3]/div[1]/div[2]/div[1]/div/h3/span')
#             #k[compet][x[4]]['Tickets']=available.text         
#             time.sleep(2)
#         except:
#             pass
#         browser.get(sites[compet])


# print(k)

In [54]:
for compet in res_unibet:
    res_unibet[compet]['cotes']= [[res_unibet[compet]['cotes'][i],res_unibet[compet]['cotes'][i+1],res_unibet[compet]['cotes'][i+2]] for i in range(len(res_unibet[compet]['cotes'])-2) if i%3==0]
for compet in res_unibet:
    for x in res_unibet[compet]['noms']:
        x[1]=x[1][1:]
for match in res_unibet['l1']['noms']:
    if match[0]=='Saint':
        match[2] = match[2][1:]
        match.pop(1)

print(res_unibet)

{'ldc': {'cotes': [['8.50', '4.70', '1.34'], ['1.34', '4.90', '7.90'], ['1.96', '3.40', '3.75'], ['5.35', '4.30', '1.53']], 'noms': [['Benfica ', 'Liverpool'], ['Manchester City ', 'Atletico Madrid'], ['Chelsea ', 'Real Madrid'], ['Villarreal ', 'Bayern Munich']], 'dates': []}, 'pl': {'cotes': [['1.12', '8.70', '17.50'], ['1.48', '4.10', '6.80'], ['11.80', '6.20', '1.22'], ['1.34', '4.85', '9.00'], ['2.37', '3.62', '2.75'], ['2.90', '3.10', '2.52'], ['1.50', '4.50', '5.40'], ['1.75', '3.68', '4.35'], ['1.48', '4.30', '6.10'], ['3.78', '3.28', '1.87'], ['2.43', '3.12', '2.70'], ['2.31', '2.95', '2.85'], ['4.10', '3.48', '1.69'], ['1.51', '3.62', '5.30'], ['4.60', '3.52', '1.60'], ['2.65', '3.08', '2.36'], ['3.02', '3.15', '2.11'], ['2.90', '3.10', '2.19'], ['2.07', '3.20', '3.05'], ['2.62', '3.00', '2.43'], ['1.96', '3.40', '3.15']], 'noms': [['Liverpool ', 'Watford'], ['Brighton ', 'Norwich City'], ['Burnley ', 'Manchester City'], ['Chelsea ', 'Brentford'], ['Leeds United ', 'Southampt

In [98]:
res = {
    'ldc' : {},
    'pl' : {},
    'l1' : {},
    'liga'  : {}
    }
sites = {'betclic': res_betclic,'winamax': res_winamax, 'unibet':res_unibet}
for site in sites:
    for comp in sites[site]:
        for i,match in enumerate(sites[site][comp]['noms']):
            if match[0][:2]+match[1][:2] not in res[comp]:
                res[comp][match[0][:2]+match[1][:2]] = {}
                res[comp][match[0][:2]+match[1][:2]]['equip1'] = match[0]
                res[comp][match[0][:2]+match[1][:2]]['equip2'] = match[1]
            try:
                res[comp][match[0][:2]+match[1][:2]][site] = sites[site][comp]['cotes'][i]
            except:
                pass





In [111]:
for compet in res:
    for match in res[compet]:
        for site in sites:
            for i in range(0,3):
                # print(i)
                # print(site)
                # print(res[compet][match])
                # print(match[site])
                try:
                    res[compet][match][site][i] = float(res[compet][match][site][i].replace(',','.'))
                except:
                    pass

for compet in res:
    print(compet,'\n' ,res[compet],'\n' )


ldc 
 {'BeLi': {'equip1': 'Benfica', 'equip2': 'Liverpool', 'betclic': [7.7, 4.3, 1.34], 'winamax': [8.5, 4.8, 1.37], 'unibet': [8.5, 4.7, 1.34]}, 'MaAt': {'equip1': 'Manchester City', 'equip2': 'Atletico Madrid', 'betclic': [1.34, 4.45, 7.25], 'winamax': [1.4, 4.75, 8.5], 'unibet': [1.34, 4.9, 7.9]}, 'ViBa': {'equip1': 'Villarreal', 'equip2': 'Bayern Munich', 'betclic': [4.95, 4.0, 1.52], 'winamax': [5.85, 4.3, 1.55], 'unibet': [5.35, 4.3, 1.53]}, 'ChRe': {'equip1': 'Chelsea', 'equip2': 'Real Madrid', 'betclic': [0.0, 0.0, 0.0], 'winamax': [2.05, 3.4, 3.75], 'unibet': [1.96, 3.4, 3.75]}} 

pl 
 {'LiWa': {'equip1': 'Liverpool', 'equip2': 'Watford', 'betclic': [1.11, 9.0, 18.0], 'winamax': [1.11, 9.5, 21.0], 'unibet': [1.12, 8.7, 17.5]}, 'BuMa': {'equip1': 'Burnley', 'equip2': 'Manchester City', 'betclic': [12.0, 6.25, 1.21], 'winamax': [14.5, 7.0, 1.21], 'unibet': [11.8, 6.2, 1.22]}, 'ChBr': {'equip1': 'Chelsea', 'equip2': 'Brentford', 'betclic': [1.33, 4.85, 8.9], 'winamax': [1.34, 5.

In [118]:
style_betclic = PatternFill(fgColor='FF0000', patternType=FILL_SOLID)
style_winamax = PatternFill(fgColor='6C6868', patternType=FILL_SOLID)
style_unibet = PatternFill(fgColor='5CB84E', patternType=FILL_SOLID)

wb = Workbook() # création du classeur
for i, compet in enumerate(res.keys()):
    if i==0:
        ws= wb.active 
        ws.title = compet
    else:
        ws = wb.create_sheet(compet)  
    ws.merge_cells('A1:K1', 'Merged Range')
    ws.cell(1,1).value= compet
    ws.cell(2,1).value= 'equip1'
    ws.cell(2,2).value= 'equip2'
    ws.merge_cells('C2:E2', 'Merged Range')
    ws.merge_cells('F2:H2', 'Merged Range')
    ws.merge_cells('I2:K2', 'Merged Range')
    ws.cell(2,3).value= 'Betclic'
    ws.cell(2,3).fill = style_betclic
    ws.cell(2,6).value= 'Winamax'
    ws.cell(2,6).fill = style_winamax
    ws.cell(2,9).value= 'Unibet'
    ws.cell(2,9).fill = style_unibet
    ws.cell(2,12).value = 'Best Bet 1'
    ws.cell(2,13).value = 'Best Bet 2'
    ws.cell(2,14).value = 'Best Bet 3'
    r,c=2,1
    for match in res[compet]:
        if 'unibet' in res[compet][match] and 'winamax' in res[compet][match] and 'betclic' in res[compet][match]:
            r+=1
            ws.cell(r,c).value=res[compet][match]['equip1']
            c+=1
            ws.cell(r,c).value=res[compet][match]['equip2']
            c+=1
            for element in res[compet][match]:
                if element != 'equip1' and element != 'equip2':
                    for cote in res[compet][match][element]:
                        if cote == '':
                            cote = 0
                            ws.cell(r,c).value= 0

                        else:    
                            ws.cell(r,c).value= cote
                        c+=1
        c=1     
    r,c = 3,12
    for match in res[compet]:
        for i in range(0,3):
            ws.cell(r,c+i).value = '= MAX('+ws.cell(r,c-9+i).coordinate+','+ws.cell(r,c-6+i).coordinate+','+ws.cell(r,c-3+i).coordinate+')'
            try:
                max_temp = max(res[compet][match]['betclic'][i],res[compet][match]['winamax'][i],res[compet][match]['unibet'][i])
                if max_temp == res[compet][match]['betclic'][i]:
                    ws.cell(r,c+i).fill = style_betclic
                if max_temp == res[compet][match]['winamax'][i]:
                    ws.cell(r,c+i).fill = style_winamax
                if max_temp == res[compet][match]['unibet'][i]:
                    ws.cell(r,c+i).fill = style_unibet
            except:
                None


        r+=1
    
    
try:
    # on sauve l'excel sous la forme xls.
    #Pour windows
    # wb.save("C:/Users/HP/OneDrive/Bureau/scrap/paris_sportif"+".xlsx")
    #Pour Mac
    wb.save("/Users/thomasbouiniere/Desktop/Scrapping/paris_sportif"+".xlsx")

except:
    print('Problème pour sauver l Excel')
browser.quit()
